## Inference

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.0+cu118 with CUDA 1108 (you have 2.5.1+cu124)
    Python  3.10.11 (you have 3.10.15)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/home/jovyan/.mlspace/envs/

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
import os
import sys
import torch
import numpy as np
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkp

The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [3]:
video_path = '/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/videoChatgpt_test/v_hFi6S_guB7I.mp4'
prompt = 'What is the appearance of the women dancers at the beginning of the video?'

output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/modeling_motion_tokenizer.py:433: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


In the beginning, the women are standing on stage and then they start dancing after opening the curtains. From the video, we can observe that they are wearing different costumes. The video does not offer any direct source for their appearance, but we can assume that they are dressed in traditional dance costumes or dresses appropriate for belly dancing performances.


In [4]:
import json
import os

import os
pwd = os.path.abspath(os.getcwd())

from tqdm import tqdm

with open("consistency_qa.json") as file:
    gt_contents = json.load(file)

    # Create the output directory if it doesn't exist
if not os.path.exists('video_gpt_test'):
    os.makedirs('video_gpt_test')

In [5]:
gt_contents[10]

{'Q1': 'What is the boy doing at the gym?',
 'Q2': 'Can you describe the equipment present at the gym?',
 'A': 'The boy is swinging between parallel bars in a well-lit gym with foam mats on the floor under various other equipment.',
 'video_name': 'v_P7Z4i1gmzjs'}

In [6]:
len(gt_contents)

499

In [ ]:


output_list = []  # List to store the output results

video_formats = ['.mp4', '.avi', '.mov', '.mkv']

output_list = []    # Iterate over each sample in the ground truth file
for i, sample in tqdm(enumerate(gt_contents)):
    video_name = sample['video_name']
    sample_set = sample
    question1 = sample['Q1']
    question2 = sample['Q2']
    answer = sample['A']

    for fmt in video_formats:  # Added this line
        temp_path = os.path.join(pwd + "/videoChatgpt_test/", f"{video_name}{fmt}")
        if os.path.exists(temp_path):
            video_path = temp_path
            break

    try:
        # Run inference on the video and add the output to the list
        output1 = runner({"video": video_path, "text_input": question1}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
        output2 = runner({"video": video_path, "text_input": question2}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
    except:
        print ("yu")

    if (i% 100 == 0):
        print (question1,'\n',  output1, '\n', answer)
        print (question2,'\n',  output2, '\n', answer)
    sample_set['pred1'] = output1
    sample_set['pred2'] = output2
    output_list.append(sample_set)

with open(os.path.join('video_gpt_test/', "consistency_answer.json"), 'w') as file:
    json.dump(output_list, file)

1it [00:27, 27.27s/it]

What equipment is visible in the gym where the boy is doing his routine? 
 The video shows two pieces of equipment in the gym where the man is doing his routine. One is the balance beam that the man is standing on after starting the routine. The other is a trampoline that he performs a backflip over. Therefore, both pieces of equipment can be seen clearly in the video. 
 There is other equipment visible in the gym like a high bar and still rings.
Can you describe the gym where the boy is performing, and what equipment is visible? 
 The gym is well-lit with light-colored walls, and various equipment is visible, including trampolines and a balance beam. 
 There is other equipment visible in the gym like a high bar and still rings.


101it [43:04, 17.54s/it]

What equipment is the man holding at the start of the video? 
 The man in the video is holding a ball, which can be clearly seen in his hand. 
 The man is holding a hammer with a handle attached to a string and he is practicing hammer throwing by swinging the hammer around him a few times before spinning and throwing it through the open end of the practice net.
Can you describe what the man is doing with the hammer in the practice net? 
 Sure! In the practice net, the man is turning around multiple times with the hammer to build momentum for the throw. He then spins the hammer around before throwing it out beyond the screen. 
 The man is holding a hammer with a handle attached to a string and he is practicing hammer throwing by swinging the hammer around him a few times before spinning and throwing it through the open end of the practice net.


475it [3:20:37, 15.99s/it]

## Validation:

In [1]:
import json

import os
import gc
import re
import sys
import csv
import time
import tqdm
import json
import numpy as np
import pandas as pd
from copy import deepcopy
from pathlib import Path
from pprint import pprint
from typing import Optional, Any, List, Union, Tuple, Dict, Callable
from collections import defaultdict, OrderedDict, Counter

import openai
import asyncio
from tqdm.asyncio import tqdm_asyncio

with open(os.path.join('video_gpt_test/', "consistency_answer.json"), 'r') as file:
    output_list = json.load(file)

In [2]:
output_list[6]

{'Q1': 'What is the man carrying in the throwing circle?',
 'Q2': 'Can you describe the hammer the man is using in the video?',
 'A': 'The man is carrying a hammer attached to a handle through a metal string and he is using it as a throwing equipment.',
 'video_name': 'v_qsEnLQ2UnEA',
 'pred1': 'The man is carrying a large ball in a throwing circle.',
 'pred2': 'The video does not provide information about the appearance of the hammer that the man is standing on. Therefore, I cannot answer your question based on the video.'}

In [3]:
from openai import OpenAI, AsyncOpenAI

client = AsyncOpenAI(api_key='YOUR_API_KEY', base_url='http://0.0.0.0:23333/v1')

In [4]:
model_names = await client.models.list()
MODEL_NAME = model_names.data[0].id
MODEL_NAME

'unsloth/llama-3-8b-Instruct'

In [10]:
import ast

scores = []
for ind, elem in enumerate(output_list):
    question1 = elem['Q1']
    answer = elem['A']
    pred1 = elem['pred1']
    question2 = elem['Q2']
    answer = elem['A']
    pred2 = elem['pred2']
    
    
    completion = await client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[
                    {
                        "role": "system",
                        "content":
                            "You are an intelligent chatbot designed for evaluating the consistency of generative outputs for similar video-based question-answer pairs. "
                            "You will be given two very similar questions, a common answer common to both the questions and predicted answers for the two questions ."
                            "Your task is to compare the predicted answers for two very similar question, with a common correct answer and determine if they are consistent. Here's how you can accomplish the task:"
                            "------"
                            "##INSTRUCTIONS: "
                            "- Focus on the consistency between the two predicted answers and the correct answer. Both predicted answers should correspond to the correct answer and to each other, and should not contain any contradictions or significant differences in the conveyed information.\n"
                            "- Both predicted answers must be consistent with each other and the correct answer, in terms of the information they provide about the video content.\n"
                            "- Consider synonyms or paraphrases as valid matches, but only if they maintain the consistency in the conveyed information.\n"
                            "- Evaluate the consistency of the two predicted answers compared to the correct answer."
                    },
                    {
                        "role": "user",
                        "content":
                            "Please evaluate the following video-based question-answer pair:\n\n"
                            f"Question 1: {question1}\n"
                            f"Question 2: {question2}\n"
                            f"Correct Answer: {answer}\n"
                            f"Predicted Answer to Question 1: {pred1}\n"
                            f"Predicted Answer to Question 2: {pred2}\n\n"
                            "Provide your evaluation only as a consistency score where the consistency score is an integer value between 0 and 5, with 5 indicating the highest level of consistency. "
                            "Please generate the response in the form of a Python dictionary string with keys 'score', where its value is the consistency score in INTEGER, not STRING."
                            "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                            "For example, your response should look like this: {''score': 4.8}."
                    }
                ]
    )

    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question1, pred1))
        print (print (question2, pred2))
        print (completion.choices[0].message.content)
    try:
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        continue

0
What equipment is visible in the gym where the boy is doing his routine? The video shows two pieces of equipment in the gym where the man is doing his routine. One is the balance beam that the man is standing on after starting the routine. The other is a trampoline that he performs a backflip over. Therefore, both pieces of equipment can be seen clearly in the video.
None
Can you describe the gym where the boy is performing, and what equipment is visible? The gym is well-lit with light-colored walls, and various equipment is visible, including trampolines and a balance beam.
None
{'score': 4}
100
What equipment is the man holding at the start of the video? The man in the video is holding a ball, which can be clearly seen in his hand.
None
Can you describe what the man is doing with the hammer in the practice net? Sure! In the practice net, the man is turning around multiple times with the hammer to build momentum for the throw. He then spins the hammer around before throwing it out b

In [16]:
print(completion.choices[0].message.content)

{'score': 5}


In [11]:
import numpy as np
np.mean(scores)

2.9218436873747495